### Bathymetry: NetCDF to .xyz

Objectives: 

- Crop a bathymetry netcdf to a selected area
- Save it as a .xyz file

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

In [2]:
# Define coordinates for the crop, filename and filepath

filename = 'GEBCO_2019.nc'
filepath = r'ondas/gustavo/GEBCO_2019'
output_filename = 'gebco_2019.xyz'

lat_i = -37
lat_f = -24
lon_i = -55
lon_f = -45


In [3]:
# Open NetCDF file with xarray
%cd ..
%cd $filepath

DS = xr.open_dataset(filename)
DS.var

/home/ceco
/home/ceco/ondas/gustavo/GEBCO_2019


<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:    (lat: 43200, lon: 86400)
Coordinates:
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
Data variables:
    elevation  (lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    title:        The GEBCO_2019 Grid - a continuous terrain model for oceans...
    institution:  On behalf of the General Bathymetric Chart of the Oceans (G...
    source:       The GEBCO_2019 Grid is the latest global bathymetric produc...
    history:      Information on the development of the data set and the sour...
    references:   DOI: 10.5285/836f016a-33be-6ddc-e053-6c86abc0788e
    comment:      The data in the GEBCO_2019 Grid should not be used for navi...
    node_offset:  1.0>

In [4]:
# Apply the crop to the selected area

DS = DS.loc[dict(lon=slice(lon_i, lon_f))]
DS = DS.loc[dict(lat=slice(lat_i, lat_f))]

In [5]:
# Separate coordinates and variables. Keep just their values

lat = DS.lat.values
lon = DS.lon.values
bat = DS.elevation.values

xr.Dataset.close(DS)

In [6]:
# Create DataFrame and a numpy array

bat_df = pd.DataFrame(data=bat, index=lat, columns=lon)
bat_array = bat_df.values


In [7]:
# Change the array to a vector

col_length = bat_array.shape[0]*bat_array.shape[1]
bat_col = np.concatenate(bat_array.reshape(col_length,1))
bat_col = bat_col.tolist()

In [8]:
# Change the indexes to vectors

lat_list = list(lat)
lon_list = list(lon)

In [9]:
# Organise the coordinates to match xyz columns

lon_col = lon_list*bat_array.shape[0]
lat_col = [ele for ele in lat_list for i in range(bat_array.shape[1])]

In [10]:
# Dictionary for creating pandas dataframe in xyz style

bat_xyz = {'X':lon_col, 'Y':lat_col, 'Z':bat_col}
bat_xyz = pd.DataFrame(bat_xyz)


In [11]:
# Save dataframe to .xyz file

bat_xyz.to_csv(output_filename, index=False, 
               header=False, sep=' ', float_format='%.7E')

In [ ]:
# Set the width of the columns of the file so they look tidy

dep = pd.read_csv(output_filename, sep='\s+', header=None, dtype='str')

columns = list(dep) 

for column in columns:
    dep[column] = dep[column].str.pad(15, side='left', fillchar=' ')

dep[0] = dep[0].str.pad(16, side='left', fillchar=' ')

import csv

dep.to_csv(output_filename, index=False, 
           header=False, sep=',')

fin = open(output_filename, "rt")
data = fin.read()
data = data.replace(',', ' ')
data = data.replace('"', '')
fin.close()

fin = open(output_filename, "wt")
fin.write(data)
fin.close()